In [5]:
import pandas as pd

# Load the dataset
df = pd.read_excel('sample_customer_database_5000_singapore.xlsx')


In [6]:
# First look at the data
print(df.head())  # First 5 rows

  Customer ID        Full Name                Email Address  Phone Number  \
0       C0001     Norma Fisher          ysullivan@yahoo.com      82421948   
1       C0002      Levi Durham            qgrimes@gmail.com      97535139   
2       C0003   Kimberly Olsen  sean96@johnston-roberts.com      71122018   
3       C0004   Matthew Davies    nguyendarrell@hotmail.com      41352560   
4       C0005  Angela Martinez    myersmitchell@johnson.com        869141   

  Date Joined     Location  Gender Loyalty Tier  \
0  2023-08-11     Tampines  Female     Platinum   
1  2022-11-24      Geylang  Female     Platinum   
2  2023-06-19     Tampines  Female     Platinum   
3  2025-04-04   Ang Mo Kio    Male       Silver   
4  2025-01-15  Bukit Batok  Female     Platinum   

                                               Notes  
0                        Together range line beyond.  
1  Language ball floor meet usually board necessary.  
2                 Support time operation wear often.  
3         

In [7]:
import numpy
import pandas

print("NumPy version:", numpy.__version__)
print("Pandas version:", pandas.__version__)


NumPy version: 1.26.4
Pandas version: 2.2.3


In [8]:
# Shape of the dataset
print("Shape of dataset:", df.shape)

Shape of dataset: (5000, 9)


In [9]:
# Columns and Data types
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   Customer ID    5000 non-null   object        
 1   Full Name      5000 non-null   object        
 2   Email Address  5000 non-null   object        
 3   Phone Number   5000 non-null   int64         
 4   Date Joined    5000 non-null   datetime64[ns]
 5   Location       5000 non-null   object        
 6   Gender         5000 non-null   object        
 7   Loyalty Tier   5000 non-null   object        
 8   Notes          5000 non-null   object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 351.7+ KB
None


In [10]:
# Check for missing values
print("Missing values per column:\n", df.isnull().sum())

Missing values per column:
 Customer ID      0
Full Name        0
Email Address    0
Phone Number     0
Date Joined      0
Location         0
Gender           0
Loyalty Tier     0
Notes            0
dtype: int64


In [11]:
# Check for duplicate rows
print("Number of duplicate rows:", df.duplicated().sum())

Number of duplicate rows: 0


In [12]:
# Unique values per column
print("Unique values per column:\n", df.nunique())

Unique values per column:
 Customer ID      5000
Full Name        4835
Email Address    4983
Phone Number     4998
Date Joined      1084
Location           27
Gender              2
Loyalty Tier        3
Notes            5000
dtype: int64


In [13]:
#text preporcessing 
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Load data
df = pd.read_excel("sample_customer_database_5000_singapore.xlsx")

# Clean Notes (for all methods)
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    filtered_tokens = [word for word in tokens if word not in stop_words]
    return ' '.join(filtered_tokens)

df['Cleaned_Notes'] = df['Notes'].apply(preprocess_text)

# Save for reuse
categorical_cols = ['Location', 'Gender', 'Loyalty Tier']

#Preview
print(df[['Notes', 'Cleaned_Notes']].head())


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lenovo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


                                               Notes  \
0                        Together range line beyond.   
1  Language ball floor meet usually board necessary.   
2                 Support time operation wear often.   
3                                  Stage plant view.   
4          Job article level others record hospital.   

                                      Cleaned_Notes  
0                        together range line beyond  
1  language ball floor meet usually board necessary  
2                 support time operation wear often  
3                                  stage plant view  
4          job article level others record hospital  


In [14]:
#model 1:Model 1: Spectral Clustering with Word2Vec for Text + One-Hot for Categorical (Separate)
# #text clustering
from gensim.models import Word2Vec
import numpy as np
from sklearn.cluster import SpectralClustering

df['Tokens'] = df['Cleaned_Notes'].apply(word_tokenize)
w2v_model = Word2Vec(df['Tokens'], vector_size=100, window=5, min_count=1, workers=4)

def average_vector(tokens, model, size=100):
    valid_tokens = [token for token in tokens if token in model.wv]
    if not valid_tokens:
        return np.zeros(size)
    return np.mean([model.wv[token] for token in valid_tokens], axis=0)

df['Text_Embeddings'] = df['Tokens'].apply(lambda x: average_vector(x, w2v_model))

X_text = np.vstack(df['Text_Embeddings'].values)
spectral = SpectralClustering(n_clusters=5, affinity='nearest_neighbors', random_state=42)
df['Text_Spectral_Label'] = spectral.fit_predict(X_text)


In [15]:
#categorical clustering
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_encoded = encoder.fit_transform(df[categorical_cols])

spectral = SpectralClustering(n_clusters=5, affinity='nearest_neighbors', random_state=42)
df['Cat_Spectral_Label'] = spectral.fit_predict(cat_encoded)


c:\Users\Lenovo\Downloads\Year 3 Major project\ML models\.venv\lib\site-packages\sklearn\manifold\_spectral_embedding.py:329: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn(


In [16]:
# Display Spectral Clustering results for TEXT
print("📊 Spectral Clustering (Text) — Cluster Counts")
print(df['Text_Spectral_Label'].value_counts())

print("\n📝 Sample Notes per Text Cluster")
for i in sorted(df['Text_Spectral_Label'].unique()):
    print(f"\n--- Cluster {i} ---")
    print(df[df['Text_Spectral_Label'] == i]['Cleaned_Notes'].head(3).to_string(index=False))

# Display Spectral Clustering results for CATEGORICAL
print("\n📊 Spectral Clustering (Categorical) — Cluster Counts")
print(df['Cat_Spectral_Label'].value_counts())

print("\n🏷️ Sample Records per Categorical Cluster")
for i in sorted(df['Cat_Spectral_Label'].unique()):
    print(f"\n--- Cluster {i} ---")
    print(df[df['Cat_Spectral_Label'] == i][['Location', 'Gender', 'Loyalty Tier']].head(3).to_string(index=False))


📊 Spectral Clustering (Text) — Cluster Counts
Text_Spectral_Label
4    2098
1    1716
0     797
2     277
3     112
Name: count, dtype: int64

📝 Sample Notes per Text Cluster

--- Cluster 0 ---
                                  stage plant view
response purpose character would partner hit an...
                        know series lay smile away

--- Cluster 1 ---
language ball floor meet usually board necessary
               support time operation wear often
                            animal exactly drive

--- Cluster 2 ---
        movie end discussion budget situation run
                                  time firm water
recently prepare scene house central baby picture

--- Cluster 3 ---
                       pm election case
bad together professional wind consider
                   reveal safe pm right

--- Cluster 4 ---
              together range line beyond
job article level others record hospital
                           part cup read

📊 Spectral Clustering (Categorical) 

In [17]:
#Model 2: HDBSCAN with UMAP Embedding for Text + Cat (Separate)
from sklearn.feature_extraction.text import TfidfVectorizer
import umap
import hdbscan

# ---------- TEXT CLUSTERING ----------
# Step 1: TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(df['Cleaned_Notes'].astype(str))

# Step 2: UMAP Dimensionality Reduction
text_embed = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42).fit_transform(tfidf_matrix)

# Step 3: HDBSCAN Clustering
df['HDBSCAN_Text_Label'] = hdbscan.HDBSCAN(min_cluster_size=10).fit_predict(text_embed)

# ---------- CATEGORICAL CLUSTERING ----------
# Step 1: UMAP on One-Hot Encoded Data
cat_embed = umap.UMAP(n_neighbors=15, min_dist=0.1, random_state=42).fit_transform(cat_encoded)

# Step 2: HDBSCAN Clustering
df['HDBSCAN_Cat_Label'] = hdbscan.HDBSCAN(min_cluster_size=10).fit_predict(cat_embed)

# ---------- Preview ----------
print(df[['Cleaned_Notes', 'HDBSCAN_Text_Label', 'HDBSCAN_Cat_Label']].head())



c:\Users\Lenovo\Downloads\Year 3 Major project\ML models\.venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Lenovo\Downloads\Year 3 Major project\ML models\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Lenovo\Downloads\Year 3 Major project\ML models\.venv\lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(
c:\Users\Lenovo\Downloads\Year 3 Major project\ML models\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Lenovo\Downloads\Year 3 Ma

                                      Cleaned_Notes  HDBSCAN_Text_Label  \
0                        together range line beyond                  78   
1  language ball floor meet usually board necessary                  21   
2                 support time operation wear often                  -1   
3                                  stage plant view                  77   
4          job article level others record hospital                  -1   

   HDBSCAN_Cat_Label  
0                147  
1                159  
2                147  
3                 13  
4                155  


c:\Users\Lenovo\Downloads\Year 3 Major project\ML models\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\Lenovo\Downloads\Year 3 Major project\ML models\.venv\lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [18]:
#updated method 3:#model 3 :Agglomerative Clustering with Gower Distance on Combined Features (With BERT)
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import OneHotEncoder
from sklearn.cluster import AgglomerativeClustering
import numpy as np

# BERT Embeddings
bert_model = SentenceTransformer('all-MiniLM-L6-v2')
bert_embeddings = bert_model.encode(df['Cleaned_Notes'].astype(str).tolist())

# Cosine Similarity from BERT
A = cosine_similarity(bert_embeddings)

# One-hot encode categorical data
categorical_cols = ['Location', 'Gender', 'Loyalty Tier']
encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
cat_encoded = encoder.fit_transform(df[categorical_cols])

# Jaccard Similarity for categorical data
intersection = np.dot(cat_encoded, cat_encoded.T)
row_sums = cat_encoded.sum(axis=1)
union = row_sums[:, None] + row_sums - intersection
B = intersection / np.maximum(union, 1e-10)

# Combine similarities
alpha = 0.5
S = alpha * A + (1 - alpha) * B
distance_matrix = 1 - S

# Agglomerative Clustering
agglo = AgglomerativeClustering(n_clusters=5, metric='precomputed', linkage='average')
df['Hybrid_Agglo_Label'] = agglo.fit_predict(distance_matrix)

# View sample
print(df[['Location', 'Gender', 'Loyalty Tier', 'Cleaned_Notes', 'Hybrid_Agglo_Label']].head())


      Location  Gender Loyalty Tier  \
0     Tampines  Female     Platinum   
1      Geylang  Female     Platinum   
2     Tampines  Female     Platinum   
3   Ang Mo Kio    Male       Silver   
4  Bukit Batok  Female     Platinum   

                                      Cleaned_Notes  Hybrid_Agglo_Label  
0                        together range line beyond                   0  
1  language ball floor meet usually board necessary                   0  
2                 support time operation wear often                   0  
3                                  stage plant view                   3  
4          job article level others record hospital                   0  


In [19]:
from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

# Text
print("Model 1 - Text (Spectral Clustering)")
print("Silhouette:", silhouette_score(X_text, df['Text_Spectral_Label']))
print("Calinski-Harabasz:", calinski_harabasz_score(X_text, df['Text_Spectral_Label']))
print("Davies-Bouldin:", davies_bouldin_score(X_text, df['Text_Spectral_Label']))

# Categorical
print("Model 1 - Cat (Spectral Clustering)")
print("Silhouette:", silhouette_score(cat_encoded, df['Cat_Spectral_Label']))
print("Calinski-Harabasz:", calinski_harabasz_score(cat_encoded, df['Cat_Spectral_Label']))
print("Davies-Bouldin:", davies_bouldin_score(cat_encoded, df['Cat_Spectral_Label']))


Model 1 - Text (Spectral Clustering)
Silhouette: 0.29619542
Calinski-Harabasz: 4043.8584294403154
Davies-Bouldin: 0.916544405713454
Model 1 - Cat (Spectral Clustering)
Silhouette: -0.059406903767330946
Calinski-Harabasz: 39.617916203354895
Davies-Bouldin: 6.033038684284049


In [20]:
# Remove noise points (-1) before calculating metrics
text_mask = df['HDBSCAN_Text_Label'] != -1
cat_mask = df['HDBSCAN_Cat_Label'] != -1

print("Model 2 - Text (HDBSCAN)")
print("Silhouette:", silhouette_score(text_embed[text_mask], df.loc[text_mask, 'HDBSCAN_Text_Label']))

print("Model 2 - Cat (HDBSCAN)")
print("Silhouette:", silhouette_score(cat_embed[cat_mask], df.loc[cat_mask, 'HDBSCAN_Cat_Label']))


Model 2 - Text (HDBSCAN)
Silhouette: 0.5065455
Model 2 - Cat (HDBSCAN)
Silhouette: 0.9877829


In [21]:
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score

# Reuse combined features (used to create similarity matrix)
# This includes BERT embeddings + one-hot categorical encoding
combined_features = np.hstack((cat_encoded, bert_embeddings))

# Get labels from Model 3
labels = df['Hybrid_Agglo_Label']

# Calinski-Harabasz Score (higher is better)
ch_score = calinski_harabasz_score(combined_features, labels)

# Davies-Bouldin Score (lower is better)
db_score = davies_bouldin_score(combined_features, labels)

print("Model 3 - Calinski-Harabasz Score:", ch_score)
print("Model 3 - Davies-Bouldin Score:", db_score)


Model 3 - Calinski-Harabasz Score: 622.1816412837292
Model 3 - Davies-Bouldin Score: 2.125900885767681
